# Country Name Map

Our main dataset contains country data from 2017 to exclude the effects of covid-19.  
We are also supplementing our main dataset with data from World Bank. Furthermore, we will also need to drop territories from our main dataset and only consider UN member states, for this we will be using the official member states csv. However a problem arises as the names of the countries in the different in the 3 datasets:
1. Difference in naming conventions
2. Changes in the name of countries over time

To resolve this problem, this notebook will create a hash map mapping the the names of countries in our main dataset to the other possible variations in the other datasets.

Data sources:
1. country_profile_variables.csv (main dataset)
2. Multiple csv files from World Bank (since all are from world bank, naming convention is the same)
3. un_member_states.csv (official list of UN member states)

In [ ]:
# imports
import pandas as pd
import numpy as np
from collections import Counter 
import json
import pprint

pp = pprint.PrettyPrinter(indent=4)

In [ ]:
# load in the main data and world bank datasets

# load in main dataset and extract country column into a series
main_data = pd.read_csv("./data/country_profile_variables_new.csv")
main_data = main_data["country"]

# load in one of the world bank csvs
# in this case we will use gni data
world_bank_countries = pd.read_csv('./data/gni_per_capita_constant_lcu.csv')
world_bank_countries = world_bank_countries["Country Name"]

In [ ]:
# we will use fuzzy string matching to speed up the process of finding same countries
# levenshtein distance for fuzzy string matching
def levenshtein_ratio_and_distance(s, t):
    # levenshtein_ratio:
    # Calculates levenshtein distance between two strings.
    # the function computes the levenshtein distance ratio of similarity between two strings

    # Initialize matrix of zeros
    rows = len(s)+1
    cols = len(t)+1
    distance = np.zeros((rows,cols),dtype = int)

    # Populate matrix of zeros with the indeces of each character of both strings
    for i in range(1, rows):
        for k in range(1,cols):
            distance[i][0] = i
            distance[0][k] = k

    # loop over the matrix to find costs  
    for col in range(1, cols):
        for row in range(1, rows):
            if s[row-1] == t[col-1]:
                cost = 0 # If the characters are the same in the two strings in a given position [i,j] then the cost is 0
            else:
                cost = 2
            distance[row][col] = min(distance[row-1][col] + 1,      # deletions costs
                                 distance[row][col-1] + 1,          # insertions cost
                                 distance[row-1][col-1] + cost)     # substitutions cost
    # Computation of the Levenshtein Distance Ratio
    Ratio = ((len(s)+len(t)) - distance[row][col]) / (len(s)+len(t))
    return Ratio





In [ ]:
# country map will be a dictionary where each key is a country from main dataset. The value is an array of 2 elements. 
# The first element is the corresponding country name in world bank data and the second element is the corresponding name in the un member state list
'''
below is an example desired endstate for country_map:

country_map = {
    "Great Britain": ["Britain", "United Kingdom"],
    "Swaziland": ["Swaziland", "Eswatini"],
    "Martinique": ["", "Martinique],
    ...
}
'''

country_map = {}

# map over all the countries and select those with high string similarity (over 0.5 for levenshtein ratio)
for index, main in main_data.items():
    country_map[main] = []
    for secondary in world_bank_countries:
        ratio = levenshtein_ratio_and_distance(main.lower(), secondary.lower())
        if ratio > 0.5:
            country_map[main].append([ratio, secondary])

# out of the countries with high similarity, choose the country with the highest ratio and drop the rest
for key in country_map:
    country = ""
    max_ratio = 0
    for item in country_map[key]:
        if item[0] > max_ratio:
            max_ratio = item[0]
            country = item[1]
    country_map[key] = country

# remove countries that have been mapped more than once
'''
hypothetical example:
Input: {"Niger": "Niger", "Nigeria": "Niger"}
Output: {"Niger": "Niger", "Nigeria": "Nigeria"}
'''
count_values = Counter(country_map.values())
values_2_or_more = []
for key, value in count_values.items():
    if key == "":
        continue
    elif value > 1:
        values_2_or_more.append(key)

compare = {}
for country in country_map:
    if country_map[country] in values_2_or_more:
        if country_map[country] not in compare:
            compare[country_map[country]] = [country]
        else:
            compare[country_map[country]].append(country)

for key in compare:
    max_country = ""
    max_ratio = 0
    for item in compare[key]:
        ratio = levenshtein_ratio_and_distance(key, item)
        if (ratio > max_ratio):
            max_ratio = ratio
            max_country = item
    compare[key] = max_country

for key in country_map:
    if key in compare:
        if country_map[key] != compare[key]:
            country_map[key] = ""

print("Check empty values and those that did not have ratio of 1: ")
for key, value in country_map.items():
    if not value or levenshtein_ratio_and_distance(key, value, True) != 1:
        print("key: ", key, "value: ", value)


In [ ]:
# map remaining countries manually
manual = {
  'Anguilla' :"",
  'Bolivia (Plurinational State of)' :"Bolivia",
  'Bonaire, Sint Eustatius and Saba' :"",
  'Congo' :"Congo, Rep.",
  'Cook Islands' :"",
  'Czechia' :"Czech Republic",
  'Democratic Republic of the Congo' :"Congo, Dem. Rep.",
  'Egypt' :"Egypt, Arab Rep.",
  'Falkland Islands (Malvinas)' :"",
  'French Guiana' :"",
  'Gambia' :"Gambia, The",
  'Guadeloupe' :"",
  'Holy See' :"",
  'Kyrgyzstan' :"Kyrgyz Republic",
  "Lao People's Democratic Republic" :"Lao PDR",
  'Martinique' :"",
  'Mayotte' :"",
  'Montserrat' :"",
  'Niue' :"",
  'Republic of Korea' :"Korea, Rep.",
  'Saint Helena' :"",
  'Saint Pierre and Miquelon' :"",
  'Slovakia' :"",
  'State of Palestine' :"",
  'Swaziland' :"Eswatini",
  'The former Yugoslav Republic of Macedonia' :"North Macedonia",
  'Tokelau' :"",
  'United Republic of Tanzania' :"Tanzania",
  'United States Virgin Islands' :"Virgin Islands (U.S.)",
  "Venezuela (Bolivarian Republic of)" :"Venezuela, RB",
  'Wallis and Futuna Islands' :"",
  'Western Sahara' :""
}

country_map.update(manual)

In [ ]:
# convert country map to values to arrays
for key in country_map:
    country_map[key] = [country_map[key]]


# load in main data to un member states map generated in excel
main_to_un_df = pd.read_csv("./data/main_to_un_names.csv")
main_to_un_df.replace(to_replace=np.nan, value='', regex=False, inplace=True)

# add un country names to country map
for key in country_map:
    country_map[key].append(main_to_un_df.loc[main_to_un_df["country"] == key, "un_country"].iloc[0])

pp.pprint(country_map)



In [ ]:
# export country map to json file
with open("./data/country_map.json", "w") as out:
    json.dump(country_map, out)